## Cannibalization
<br>
This notebook identifies possible <strong>keyword cannibalization issues</strong> using Google Search Console data. <br>  
<br>
    @author: Natzir Turrado: Technical SEO / Data Scientist. <a href="https://twitter.com/natzir9">Twitter > @natzir9</a>


Importing libraries

In [ ]:
import pandas as pd
import re
import datetime
from google.colab import widgets
#import qgrid #for dynamic grids in jupiter notebook
from collections import defaultdict
from dateutil import relativedelta
import httplib2
from apiclient import errors
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow
import requests
from bs4 import BeautifulSoup
from google.colab import files

Insert here your Google **CLIENT_ID**, **CLIENT_SECRET** & your Search Console **SITE PROPERTY**. 
<ul><li><a href="https://console.developers.google.com/flows/enableapi?apiid=searchconsole.googleapis.com&credential=client_key">Create your API credentintials</a> (OAuth 2.0 Client IDs)</li></ul>

In [ ]:
CLIENT_ID = '758312227186-r8r2s7jhp481solkm4b0t59aah292sha.apps.googleusercontent.com'
CLIENT_SECRET = '2mP2K2fY9baJ1Y227xgQ9U4c'
site = 'https://www.zankyou.es/' #in the same way as it is in the Search Console property

Insert here the **date range** (last 3 month of SC data by default)



In [ ]:
end_date = datetime.date.today()
start_date = end_date - relativedelta.relativedelta(months=3)

Google Search Console API call. 
<ul><li><a href="https://developers.google.com/webmaster-tools/search-console-api-original/v3/quickstart/quickstart-python">Quickstart: Run a Search Console App in Python</a></li>
<li><a href="https://developers.google.com/apis-explorer/#p/webmasters/v3/">Search Console API Explorer</a></li>
</ul>

In [ ]:
OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly'
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

# Run through the OAuth flow and retrieve credentials
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, redirect_uri=REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print ('Go to the following link in your browser: ' + authorize_url)
code = input('Enter verification code: ').strip()
credentials = flow.step2_exchange(code)

# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()
http = credentials.authorize(http)

webmasters_service = build('webmasters', 'v3', http=http)

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=758312227186-r8r2s7jhp481solkm4b0t59aah292sha.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&access_type=offline&response_type=code


In [ ]:
def execute_request(service, property_uri, request):
    return service.searchanalytics().query(siteUrl=property_uri, body=request).execute()

request = {
    'startDate': datetime.datetime.strftime(start_date,"%Y-%m-%d"),
    'endDate': datetime.datetime.strftime(end_date,'%Y-%m-%d'),
    'dimensions': ['page','query'],
    'rowLimit': 25000 #up to 25.000 urls
}

#Adding a device filter to request
device_category = input('Enter device category: MOBILE, DESKTOP or TABLET (leave it blank for all devices): ').strip()
if device_category:
    request_page['dimensionFilterGroups'] = [{'filters':[{'dimension':'device','expression':device_category}]}]
    request_query['dimensionFilterGroups'] = [{'filters': [{'dimension': 'device', 'expression': device_category}]}]
    request_date['dimensionFilterGroups'] = [{'filters': [{'dimension': 'device', 'expression': device_category}]}]
else:
    device_category = 'ALL Devices'

#Request to SC API
response = execute_request(webmasters_service, site, request)

Enter device category: MOBILE, DESKTOP or TABLET (leave it blank for all devices): 


<strong>Parsing the JSON returned</strong>

In [ ]:
scDict = defaultdict(list)

for row in response['rows']:
    scDict['page'].append(row['keys'][0] or 0)
    scDict['query'].append(row['keys'][1] or 0)
    scDict['clicks'].append(row['clicks'] or 0)
    scDict['ctr'].append(row['ctr'] or 0)
    scDict['impressions'].append(row['impressions'] or 0)
    scDict['position'].append(row['position'] or 0)

<strong>DataFrame of Search Console data<strong>


In [ ]:
df = pd.DataFrame(data = scDict)

df['clicks'] = df['clicks'].astype('int')
df['ctr'] = df['ctr']*100
df['impressions'] = df['impressions'].astype('int')
df['position'] = df['position'].round(2)
df.sort_values('clicks',inplace=True,ascending=False)
df

,page,query,clicks,ctr,impressions,position
0,https://www.zankyou.es/p/trucos-para-escribir-...,votos matrimoniales,5394,13.637053,39554,4.82
1,https://www.zankyou.es/p/los-20-regalos-que-to...,regalos para mujer,5327,12.830270,41519,1.39
2,https://www.zankyou.es/p/los-20-regalos-que-to...,regalos para mi novia,4612,11.043268,41763,2.07
3,https://www.zankyou.es/,zankyou,3868,50.456561,7666,2.11
4,https://www.zankyou.es/p/los-20-regalos-que-to...,regalos para mujeres,2549,12.636328,20172,1.41
...,...,...,...,...,...,...
24613,https://www.zankyou.es/f/almacenes-aragon-421372,almacenes aragon ibiza,2,1.176471,170,7.21
24612,https://www.zankyou.es/f/alma-novias-484223,alma novias 2020 precios,2,100.000000,2,14.00
24611,https://www.zankyou.es/f/alma-makeup-625912,alma makeup,2,40.000000,5,2.00
24610,https://www.zankyou.es/f/alma-estetica-617664,alma estetica,2,0.453515,441,13.95


<strong>Cleaning the DataFrame and sorting it by query</strong>
<ul>
   <li>Excludes page one. According to Mozcast, the average page one has only 8 listings, so we only look for URLs beyond this position (this behaviour could by changed in 'SERP_result').</li>
   <li>Excludes branded queries (set yours in 'branded_queries' )</li>
   <li>Excludes unique queries</li>    
<ul>

In [ ]:
SERP_results = 8 #insert here your prefered value for SERP results
branded_queries = 'zankyou|zank you|zanyou|sankyou|zankyuo|zankyu' # insert here your branded keywords

In [ ]:
df_cannibalized = df[df['position'] > SERP_results] # excluding first page results
df_cannibalized

,page,query,clicks,ctr,impressions,position
11,https://www.zankyou.es/p/nombres-de-aniversari...,aniversario de bodas,1712,1.674148,102261,8.63
36,https://www.zankyou.es/p/7-curiosidades-que-te...,anillos de compromiso,644,0.421069,152944,8.33
73,https://www.zankyou.es/p/como-organizar-una-or...,despedida de soltera,447,0.458814,97425,8.27
93,https://www.zankyou.es/p/62-canciones-de-regga...,reggaeton antiguo,383,6.638932,5769,19.09
95,https://www.zankyou.es/f/lamarr-741036,lamarr alicante,376,3.107952,12098,9.80
...,...,...,...,...,...,...
24620,https://www.zankyou.es/f/alqueria-kukala-grupo...,grupo la cartuja,2,3.076923,65,11.31
24615,https://www.zankyou.es/f/almacenes-aragon-421372,muebles aragón ibiza,2,3.448276,58,8.05
24612,https://www.zankyou.es/f/alma-novias-484223,alma novias 2020 precios,2,100.000000,2,14.00
24610,https://www.zankyou.es/f/alma-estetica-617664,alma estetica,2,0.453515,441,13.95


In [ ]:
df_cannibalized = df_cannibalized[~df_cannibalized['query'].str.contains(branded_queries, regex=True)] # excluding branded keywords
df_cannibalized = df_cannibalized[df_cannibalized.duplicated(subset=['query'], keep=False)] # getting the duplicate queries
df_cannibalized

,page,query,clicks,ctr,impressions,position
36,https://www.zankyou.es/p/7-curiosidades-que-te...,anillos de compromiso,644,0.421069,152944,8.33
518,https://www.zankyou.es/p/como-organizar-una-bo...,bodas sencillas,106,2.884354,3675,8.71
548,https://www.zankyou.es/p/vestidos-de-fiesta-en...,vestidos de fiesta,103,1.426000,7223,9.82
594,https://www.zankyou.es/p/como-triunfar-con-tu-...,vestidos de fiesta para bodas,96,1.187237,8086,8.95
671,https://www.zankyou.es/p/vestidos-de-fiesta-en...,vestidos fiesta,87,2.557319,3402,8.64
...,...,...,...,...,...,...
24913,https://www.zankyou.es/f/carla-ruiz-online-8676,carla ruiz,2,25.000000,8,12.50
24914,https://www.zankyou.es/f/carla-ruiz-sevilla-8681,carla ruiz,2,2.666667,75,12.16
24552,https://www.zankyou.es/f/a-tipica-6961,a-tipica,2,0.668896,299,8.82
24442,https://www.zankyou.es/alquiler-trajes-novio/m...,alquiler de trajes para bodas,2,1.851852,108,8.75


In [ ]:
df_cannibalized.set_index(['query'],inplace=True) # making queries the index of the DF
df_cannibalized

,page,clicks,ctr,impressions,position
query,,,,,
anillos de compromiso,https://www.zankyou.es/p/7-curiosidades-que-te...,644,0.421069,152944,8.33
bodas sencillas,https://www.zankyou.es/p/como-organizar-una-bo...,106,2.884354,3675,8.71
vestidos de fiesta,https://www.zankyou.es/p/vestidos-de-fiesta-en...,103,1.426000,7223,9.82
vestidos de fiesta para bodas,https://www.zankyou.es/p/como-triunfar-con-tu-...,96,1.187237,8086,8.95
vestidos fiesta,https://www.zankyou.es/p/vestidos-de-fiesta-en...,87,2.557319,3402,8.64
...,...,...,...,...,...
carla ruiz,https://www.zankyou.es/f/carla-ruiz-online-8676,2,25.000000,8,12.50
carla ruiz,https://www.zankyou.es/f/carla-ruiz-sevilla-8681,2,2.666667,75,12.16
a-tipica,https://www.zankyou.es/f/a-tipica-6961,2,0.668896,299,8.82


In [ ]:
df_cannibalized.sort_index(inplace=True) # queries in alphabetical order to help identifying cannibalization
df_cannibalized

,page,clicks,ctr,impressions,position
query,,,,,
a-tipica,https://www.zankyou.es/f/a-tipica-6961,2,0.668896,299,8.82
a-tipica,https://www.zankyou.es/f/a-tipica-shop-382131,4,1.036269,386,10.09
acus,https://www.zankyou.es/f/acus-complementos-581436,23,0.205064,11216,9.11
acus,https://www.zankyou.es/f/acus-complementos-524672,4,0.108784,3677,10.24
acus complementos,https://www.zankyou.es/f/acus-complementos-524672,15,0.174846,8579,9.54
...,...,...,...,...,...
wedding planner,https://www.zankyou.es/p/18-preguntas-que-debe...,10,0.265393,3768,13.25
wedding planner,https://www.zankyou.es/p/6-cosas-que-olvidaras...,6,0.485044,1237,13.38
zapatos de novia,https://www.zankyou.es/zapatos-novias/valencia,6,2.884615,208,9.43


In [ ]:
df_cannibalized.reset_index(inplace=True) # resetting the index to have the queries as a column again
df_cannibalized

,query,page,clicks,ctr,impressions,position
0,a-tipica,https://www.zankyou.es/f/a-tipica-6961,2,0.668896,299,8.82
1,a-tipica,https://www.zankyou.es/f/a-tipica-shop-382131,4,1.036269,386,10.09
2,acus,https://www.zankyou.es/f/acus-complementos-581436,23,0.205064,11216,9.11
3,acus,https://www.zankyou.es/f/acus-complementos-524672,4,0.108784,3677,10.24
4,acus complementos,https://www.zankyou.es/f/acus-complementos-524672,15,0.174846,8579,9.54
...,...,...,...,...,...,...
307,wedding planner,https://www.zankyou.es/p/18-preguntas-que-debe...,10,0.265393,3768,13.25
308,wedding planner,https://www.zankyou.es/p/6-cosas-que-olvidaras...,6,0.485044,1237,13.38
309,zapatos de novia,https://www.zankyou.es/zapatos-novias/valencia,6,2.884615,208,9.43
310,zapatos de novia,https://www.zankyou.es/zapatos-novias/sevilla,8,4.444444,180,10.12


<strong>Scraping URLs and Adding Titles and Meta Descriptions to the DataFrame</strong>

In [ ]:
def get_meta(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    title = soup.find('title').get_text()
    meta = soup.select('meta[name="description"]')[0].attrs["content"] 
    return title, meta

df_cannibalized['title'],df_cannibalized['meta'] = zip(*df_cannibalized['page'].apply(get_meta))
df_cannibalized

,query,page,clicks,ctr,impressions,position,title,meta
0,a-tipica,https://www.zankyou.es/f/a-tipica-6961,2,0.668896,299,8.82,"A-típica - Opiniones, Fotos y Teléfono","Wedding planners - A-típica. Packs para bodas,..."
1,a-tipica,https://www.zankyou.es/f/a-tipica-shop-382131,4,1.036269,386,10.09,"A-típica Shop - Opiniones, Fotos y Teléfono",Decoración para su boda - A-típica Shop. Preci...
2,acus,https://www.zankyou.es/f/acus-complementos-581436,23,0.205064,11216,9.11,"ACUS Complementos Online - Opiniones, Fotos y ...",Tocados y Complementos de novia - ACUS Complem...
3,acus,https://www.zankyou.es/f/acus-complementos-524672,4,0.108784,3677,10.24,"ACUS Complementos - Opiniones, Fotos y Teléfono",Joyerías para bodas - ACUS Complementos. Packs...
4,acus complementos,https://www.zankyou.es/f/acus-complementos-524672,15,0.174846,8579,9.54,"ACUS Complementos - Opiniones, Fotos y Teléfono",Joyerías para bodas - ACUS Complementos. Packs...
...,...,...,...,...,...,...,...,...
307,wedding planner,https://www.zankyou.es/p/18-preguntas-que-debe...,10,0.265393,3768,13.25,Cómo elegir wedding planner para mi boda en 5 ...,El wedding planner es uno de los profesionales...
308,wedding planner,https://www.zankyou.es/p/6-cosas-que-olvidaras...,6,0.485044,1237,13.38,10 cosas que olvidarás si no cuentas con un we...,"Aunque no lo creas así, tener un organizador d..."
309,zapatos de novia,https://www.zankyou.es/zapatos-novias/valencia,6,2.884615,208,9.43,Zapatos para novias en Valencia,Selección de Zapatos para novias en Valencia c...
310,zapatos de novia,https://www.zankyou.es/zapatos-novias/sevilla,8,4.444444,180,10.12,Zapatos para novias en Sevilla,Selección de Zapatos para novias en Sevilla co...


<strong>Creating a dynamic grid to analyse the data</strong> (qgrid widgets do not work in Colab)


In [ ]:
#grid = qgrid.show_grid(df_cannibalized, show_toolbar=True)
#grid
%load_ext google.colab.data_table
df_cannibalized

,query,page,clicks,ctr,impressions,position,title,meta
0,a-tipica,https://www.zankyou.es/f/a-tipica-6961,2,0.668896,299,8.82,"A-típica - Opiniones, Fotos y Teléfono","Wedding planners - A-típica. Packs para bodas,..."
1,a-tipica,https://www.zankyou.es/f/a-tipica-shop-382131,4,1.036269,386,10.09,"A-típica Shop - Opiniones, Fotos y Teléfono",Decoración para su boda - A-típica Shop. Preci...
2,acus,https://www.zankyou.es/f/acus-complementos-581436,23,0.205064,11216,9.11,"ACUS Complementos Online - Opiniones, Fotos y ...",Tocados y Complementos de novia - ACUS Complem...
3,acus,https://www.zankyou.es/f/acus-complementos-524672,4,0.108784,3677,10.24,"ACUS Complementos - Opiniones, Fotos y Teléfono",Joyerías para bodas - ACUS Complementos. Packs...
4,acus complementos,https://www.zankyou.es/f/acus-complementos-524672,15,0.174846,8579,9.54,"ACUS Complementos - Opiniones, Fotos y Teléfono",Joyerías para bodas - ACUS Complementos. Packs...
...,...,...,...,...,...,...,...,...
307,wedding planner,https://www.zankyou.es/p/18-preguntas-que-debe...,10,0.265393,3768,13.25,Cómo elegir wedding planner para mi boda en 5 ...,El wedding planner es uno de los profesionales...
308,wedding planner,https://www.zankyou.es/p/6-cosas-que-olvidaras...,6,0.485044,1237,13.38,10 cosas que olvidarás si no cuentas con un we...,"Aunque no lo creas así, tener un organizador d..."
309,zapatos de novia,https://www.zankyou.es/zapatos-novias/valencia,6,2.884615,208,9.43,Zapatos para novias en Valencia,Selección de Zapatos para novias en Valencia c...
310,zapatos de novia,https://www.zankyou.es/zapatos-novias/sevilla,8,4.444444,180,10.12,Zapatos para novias en Sevilla,Selección de Zapatos para novias en Sevilla co...


<strong>Saving to CSV</strong>

In [ ]:
#df_cannibalized.to_csv('cannibalized.csv')
#files.download('cannibalized.csv')